In [ ]:
# github of TResNet: "https://github.com/Alibaba-MIIL/TResNet"
from src_files.models.tresnet.tresnet import TResnetL

from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
from google.colab import drive

In [ ]:
import torchvision
import torchvision.transforms as transforms
n = 1
for i in ["/content/drive/MyDrive/CS172B Final Project/Data_after_train_test_val_split/test", "/content/drive/MyDrive/CS172B Final Project/Data_after_train_test_val_split/train", "/content/drive/MyDrive/CS172B Final Project/Data_after_train_test_val_split/val"]:
  exec(f"dataset{n} = torchvision.datasets.ImageFolder(i, transforms.Compose([\
    transforms.Resize(256),\
    transforms.CenterCrop(224),\
    transforms.ToTensor(),\
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),\
]),is_valid_file = lambda x: '.jpg' in x)")
  n+=1

test_data = dataset1
train_data = dataset2
val_data = dataset3

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print(device)

In [ ]:
# print(dataset1.class_to_idx)
# print(dataset2.class_to_idx)

temp = {}
for i,j in enumerate(sorted(list(set(dataset1.class_to_idx.keys()).union(set(dataset2.class_to_idx.keys())).union(set(dataset3.class_to_idx.keys()))))):
    temp[j] = i
temp
test_data.class_to_idx=temp
train_data.class_to_idx=temp
val_data.class_to_idx=temp

In [ ]:
epoch = 200
num_classes = len(temp)
batch_size = 32
learning_rate = 0.002
test_num = int((11569//batch_size + 1) * 0.2)
train_num = (11569//batch_size + 1) - test_num

In [ ]:
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = True)
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_data, batch_size = batch_size, shuffle = True)

In [ ]:
model = TResnetM({'num_classes':len(temp)}).to(device)

In [ ]:
# Train the model and print the loss per epoch (5 points)
info_list = []
val_correct = []
test_correct = []
total_correct = 0
for j in range(epoch):
    train_correct = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        pred = model(images)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()

        predictions = pred.max(dim=1).indices
        train_correct += torch.eq(predictions, labels).sum().item()
        total_correct += torch.eq(predictions, labels).sum().item()
        info_list.append((train_correct, loss))

    
    val_total_correct = 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(val_loader):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            pred = model( images )
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
                
            predictions = pred.max(dim=1).indices
            correct = torch.eq(predictions, labels).sum().item()
            val_total_correct += correct
            val_correct.append((j,i,correct))
            torch.cuda.empty_cache()

    accuracy = train_correct / (len(train_loader) *batch_size)
    
    print(f'epoch: {j+1}', "Training Accuracy: ", accuracy, "Loss: ", loss.item(),
    f"val correct: {val_total_correct}")


In [ ]:
torch.save(model.state_dict(), './TResNet_saved')

  We are using the validation data as the testing data instead of original testing data. The reason can be found in the TResNet part of the final report